<a href="https://colab.research.google.com/github/tanvithakur22/BNPL_Risk-Adjusted-Profitability-Credit-Strategy-Simulation/blob/main/bnpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("bnpl_portfolio_dataset.csv")

df.head()


,user_id,age,income,credit_score,loan_amount,tenure_months,interest_rate,merchant_commission_rate,acquisition_cost,defaulted,revenue,loss,net_profit
0,BNPL_000001,49,118694,406,23575,3,0.18,0.03,388,0,1768.12,0,1380.12
1,BNPL_000002,39,52194,758,16423,3,0.18,0.03,572,0,1231.72,0,659.72
2,BNPL_000003,44,56090,608,2769,3,0.18,0.03,543,0,207.67,0,-335.33
3,BNPL_000004,22,20311,576,19568,3,0.18,0.03,513,0,1467.60,0,954.60
4,BNPL_000005,45,40658,810,26233,3,0.18,0.03,387,0,1967.47,0,1580.47


Portfolio Risk Segmentation.

In [2]:
def credit_bucket(score):
    if score >= 750:
        return "Super Prime"
    elif score >= 650:
        return "Prime"
    elif score >= 550:
        return "Near Prime"
    else:
        return "Subprime"

df["credit_segment"] = df["credit_score"].apply(credit_bucket)

df.head()


,user_id,age,income,credit_score,loan_amount,tenure_months,interest_rate,merchant_commission_rate,acquisition_cost,defaulted,revenue,loss,net_profit,credit_segment
0,BNPL_000001,49,118694,406,23575,3,0.18,0.03,388,0,1768.12,0,1380.12,Subprime
1,BNPL_000002,39,52194,758,16423,3,0.18,0.03,572,0,1231.72,0,659.72,Super Prime
2,BNPL_000003,44,56090,608,2769,3,0.18,0.03,543,0,207.67,0,-335.33,Near Prime
3,BNPL_000004,22,20311,576,19568,3,0.18,0.03,513,0,1467.60,0,954.60,Near Prime
4,BNPL_000005,45,40658,810,26233,3,0.18,0.03,387,0,1967.47,0,1580.47,Super Prime


In [3]:
segment_analysis = df.groupby("credit_segment").agg(
    total_users = ("user_id", "count"),
    default_rate = ("defaulted", "mean"),
    avg_revenue = ("revenue", "mean"),
    avg_loss = ("loss", "mean"),
    avg_net_profit = ("net_profit", "mean"),
    total_net_profit = ("net_profit", "sum")
).reset_index()

segment_analysis


,credit_segment,total_users,default_rate,avg_revenue,avg_loss,avg_net_profit,total_net_profit
0,Near Prime,8894,0.121205,1201.665992,1941.415898,-1140.641137,-10144862.27
1,Prime,9257,0.057794,1194.509308,954.460732,-158.639812,-1468528.74
2,Subprime,22822,0.247130,1200.953478,3946.721672,-3144.952877,-71774114.56
3,Super Prime,9027,0.019497,1202.347258,313.689376,487.812572,4403484.09


In [4]:
segment_analysis["default_rate"] = segment_analysis["default_rate"] * 100

segment_analysis


,credit_segment,total_users,default_rate,avg_revenue,avg_loss,avg_net_profit,total_net_profit
0,Near Prime,8894,12.120531,1201.665992,1941.415898,-1140.641137,-10144862.27
1,Prime,9257,5.779410,1194.509308,954.460732,-158.639812,-1468528.74
2,Subprime,22822,24.712996,1200.953478,3946.721672,-3144.952877,-71774114.56
3,Super Prime,9027,1.949706,1202.347258,313.689376,487.812572,4403484.09


Approval Threshold Optimization

In [5]:
def evaluate_threshold(threshold):
    filtered_df = df[df["credit_score"] >= threshold]

    total_users = len(filtered_df)
    total_profit = filtered_df["net_profit"].sum()
    avg_profit = filtered_df["net_profit"].mean()
    default_rate = filtered_df["defaulted"].mean() * 100

    return {
        "threshold": threshold,
        "approved_users": total_users,
        "total_profit": round(total_profit, 2),
        "avg_profit_per_user": round(avg_profit, 2),
        "default_rate (%)": round(default_rate, 2)
    }


In [6]:
results = []

for t in [550, 650, 700, 750]:
    results.append(evaluate_threshold(t))

threshold_comparison = pd.DataFrame(results)

threshold_comparison


,threshold,approved_users,total_profit,avg_profit_per_user,default_rate (%)
0,550,27178,-7209906.92,-265.28,6.58
1,650,18284,2934955.35,160.52,3.89
2,700,13692,3592999.17,262.42,3.30
3,750,9027,4403484.09,487.81,1.95


ASSIGN INTEREST ACCORDING TO RISK

In [7]:
def assign_interest(segment):
    if segment == "Super Prime":
        return 0.18
    elif segment == "Prime":
        return 0.20
    elif segment == "Near Prime":
        return 0.24
    else:
        return 0.30

df["adjusted_interest_rate"] = df["credit_segment"].apply(assign_interest)


In [8]:
df["new_interest_income"] = df["loan_amount"] * df["adjusted_interest_rate"] * (df["tenure_months"] / 12)

df["new_revenue"] = df["new_interest_income"] + (df["loan_amount"] * df["merchant_commission_rate"])


In [9]:
df["new_net_profit"] = df["new_revenue"] - df["loss"] - df["acquisition_cost"]


In [10]:
new_segment_analysis = df.groupby("credit_segment").agg(
    total_users=("user_id", "count"),
    default_rate=("defaulted", "mean"),
    avg_new_profit=("new_net_profit", "mean"),
    total_new_profit=("new_net_profit", "sum")
).reset_index()

new_segment_analysis["default_rate"] *= 100

new_segment_analysis


,credit_segment,total_users,default_rate,avg_new_profit,total_new_profit
0,Near Prime,8894,12.120531,-900.306825,-8.007329e+06
1,Prime,9257,5.779410,-79.004809,-7.313475e+05
2,Subprime,22822,24.712996,-2664.570242,-6.081082e+07
3,Super Prime,9027,1.949706,487.813584,4.403493e+06


loan adjusment vs loan and interest adjustment

In [11]:
def adjust_loan(segment, loan):
    if segment == "Super Prime":
        return loan
    elif segment == "Prime":
        return loan * 0.9
    elif segment == "Near Prime":
        return loan * 0.7
    else:
        return loan * 0.5

df["adjusted_loan_amount"] = df.apply(
    lambda row: adjust_loan(row["credit_segment"], row["loan_amount"]), axis=1
)


In [12]:
df["loan_only_interest_income"] = df["adjusted_loan_amount"] * 0.18 * (df["tenure_months"] / 12)

df["loan_only_revenue"] = df["loan_only_interest_income"] + \
                          (df["adjusted_loan_amount"] * df["merchant_commission_rate"])


In [13]:
df["loan_only_loss"] = df["adjusted_loan_amount"] * df["defaulted"]


In [14]:
df["loan_only_net_profit"] = df["loan_only_revenue"] - \
                             df["loan_only_loss"] - \
                             df["acquisition_cost"]


In [15]:
loan_only_analysis = df.groupby("credit_segment").agg(
    avg_profit=("loan_only_net_profit", "mean"),
    total_profit=("loan_only_net_profit", "sum"),
    default_rate=("defaulted", "mean")
).reset_index()

loan_only_analysis["default_rate"] *= 100

loan_only_analysis


,credit_segment,avg_profit,total_profit,default_rate
0,Near Prime,-918.715581,-8.171056e+06,12.120531
1,Prime,-182.643795,-1.690734e+06,5.779410
2,Subprime,-1772.068510,-4.044215e+07,24.712996
3,Super Prime,487.813584,4.403493e+06,1.949706


In [16]:
df["combined_interest_income"] = df["adjusted_loan_amount"] * \
                                 df["adjusted_interest_rate"] * \
                                 (df["tenure_months"] / 12)

df["combined_revenue"] = df["combined_interest_income"] + \
                         (df["adjusted_loan_amount"] * df["merchant_commission_rate"])


In [17]:
df["combined_loss"] = df["adjusted_loan_amount"] * df["defaulted"]


In [18]:
df["combined_net_profit"] = df["combined_revenue"] - \
                            df["combined_loss"] - \
                            df["acquisition_cost"]


In [19]:
combined_analysis = df.groupby("credit_segment").agg(
    avg_profit=("combined_net_profit", "mean"),
    total_profit=("combined_net_profit", "sum"),
    default_rate=("defaulted", "mean")
).reset_index()

combined_analysis["default_rate"] *= 100

combined_analysis


,credit_segment,avg_profit,total_profit,default_rate
0,Near Prime,-750.482194,-6.674789e+06,12.120531
1,Prime,-110.973174,-1.027279e+06,5.779410
2,Subprime,-1531.877598,-3.496051e+07,24.712996
3,Super Prime,487.813584,4.403493e+06,1.949706


In [ ]:
| Segment              | Decision                                   |
| -------------------- | ------------------------------------------ |
| 750+ (Super Prime)   | Approve full loan, base interest (18%)     |
| 650–749 (Prime)      | Approve full loan, moderate interest (20%) |
| 550–649 (Near Prime) | Approve 50% loan, high interest (28%)      |
| <550 (Subprime)      | Reject completely                          |


In [20]:
df["approved"] = df["credit_score"] >= 550


In [21]:
def final_loan_policy(row):
    if row["credit_score"] >= 750:
        return row["loan_amount"]
    elif row["credit_score"] >= 650:
        return row["loan_amount"]
    elif row["credit_score"] >= 550:
        return row["loan_amount"] * 0.5
    else:
        return 0

df["final_loan_amount"] = df.apply(final_loan_policy, axis=1)


In [22]:
def final_interest_policy(score):
    if score >= 750:
        return 0.18
    elif score >= 650:
        return 0.20
    elif score >= 550:
        return 0.28
    else:
        return 0

df["final_interest_rate"] = df["credit_score"].apply(final_interest_policy)


In [25]:
df["final_interest_income"] = df["final_loan_amount"] * \
                              df["final_interest_rate"] * \
                              (df["tenure_months"] / 12)

df["final_revenue"] = df["final_interest_income"] + \
                      (df["final_loan_amount"] * df["merchant_commission_rate"])


In [26]:
df["final_loss"] = df["final_loan_amount"] * df["defaulted"]


In [27]:
df["final_net_profit"] = df["final_revenue"] - \
                         df["final_loss"] - \
                         df["acquisition_cost"]


In [28]:
final_portfolio = df[df["credit_score"] >= 550]

final_summary = {
    "approved_users": len(final_portfolio),
    "total_profit": round(final_portfolio["final_net_profit"].sum(), 2),
    "avg_profit_per_user": round(final_portfolio["final_net_profit"].mean(), 2),
    "default_rate (%)": round(final_portfolio["defaulted"].mean() * 100, 2)
}

final_summary


{'approved_users': 27178,
 'total_profit': np.float64(-1401774.3),
 'avg_profit_per_user': np.float64(-51.58),
 'default_rate (%)': np.float64(6.58)}

Here’s what we learned:

750+ only → High profit, very low volume

Approving everyone → High volume, heavy loss

Loan reduction + higher interest → Helped, but still slightly negative

You're only losing ₹51 per user → Very close to break-even

So you don’t need extreme changes.

Portfolio is thin margin.

Even with controlled default (6.58%),
acquisition cost (CAC) eats the margin.

You are very close to break-even.

increase merchant charges

In [29]:
df["final_revenue_updated"] = df["final_interest_income"] + \
                              (df["final_loan_amount"] * 0.04)

df["final_net_profit_updated"] = df["final_revenue_updated"] - \
                                 df["final_loss"] - \
                                 df["acquisition_cost"]

updated_portfolio = df[df["credit_score"] >= 550]

{
    "total_profit": round(updated_portfolio["final_net_profit_updated"].sum(), 2),
    "avg_profit_per_user": round(updated_portfolio["final_net_profit_updated"].mean(), 2)
}


{'total_profit': np.float64(2232224.9),
 'avg_profit_per_user': np.float64(82.13)}